In [4]:
import cv2
from cv2 import aruco
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
import math as m
import random
import numpy.linalg as lin
import numpy as np
from numpy import genfromtxt

In [23]:
camera_matrix = genfromtxt('const/mtx.csv', delimiter=',')
dist_coeffs = genfromtxt('const/dst.csv', delimiter=',')
squareLength, markerLength = 23, 18.5
aruco_dict = aruco.Dictionary_get(aruco.DICT_5X5_100)
arucoParams = aruco.DetectorParameters_create()

img = cv2.imread("iphone/KakaoTalk_20230728_085933013.jpg")
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(
    camera_matrix, dist_coeffs, (w, h), 1, (w, h))

dst = cv2.undistort(img, camera_matrix,
                    dist_coeffs, None, newcameramtx)

corners, ids, rejectedImgPoints = aruco.detectMarkers(dst, aruco_dict,
                                                        parameters=arucoParams)
diamondCorners, diamondIds = aruco.detectCharucoDiamond(dst, corners, ids, squareLength / markerLength,
                                                                            diamondCorners=None, diamondIds=None,
                                                                            cameraMatrix=None, distCoeffs=None)
rvecs, tvecs, _objPoints = aruco.estimatePoseSingleMarkers(diamondCorners, markerLength, newcameramtx,
                                                            dist_coeffs, rvecs=None, tvecs=None,
                                                            _objPoints=None)

aruco_coord = np.reshape(diamondCorners, (4, 2))
im_with_charuco_board = aruco.drawAxis(dst, newcameramtx, dist_coeffs, rvecs[0], tvecs[0],
                                        10)
im_with_charuco_board = cv2.circle(im_with_charuco_board, (int(
    aruco_coord[0, 0]), int(aruco_coord[0, 1])), 15, (255, 255, 255), -1)
im_with_charuco_board = cv2.circle(im_with_charuco_board, (int(
    aruco_coord[1, 0]), int(aruco_coord[1, 1])), 15, (255, 0, 0), -1)
im_with_charuco_board = cv2.circle(im_with_charuco_board, (int(
    aruco_coord[2, 0]), int(aruco_coord[2, 1])), 15, (0, 255, 0), -1)
im_with_charuco_board = cv2.circle(im_with_charuco_board, (int(
    aruco_coord[3, 0]), int(aruco_coord[3, 1])), 15, (0, 0, 255), -1)

fig = cv2.resize(im_with_charuco_board, (0,0), fx=0.3, fy=0.3)
cv2.imshow('fig', fig)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [32]:
def isRotationMatrix(R):
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype=R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6

def rot2Eul(R):

    assert (isRotationMatrix(R))
    sy = m.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])

    singular = sy < 1e-6

    if not singular:
        x = m.atan2(R[2, 1], R[2, 2])
        y = m.atan2(-R[2, 0], sy)
        z = m.atan2(R[1, 0], R[0, 0])
    else:
        x = m.atan2(-R[1, 2], R[1, 1])
        y = m.atan2(-R[2, 0], sy)
        z = 0

    pitch = x/m.pi*180
    yaw = y/m.pi*180
    roll = z/m.pi*180

    return np.array([pitch, yaw, roll])

def eul2rotm(roll,yaw,pitch):
    roll = m.pi/180 * roll
    yaw = m.pi/180 * yaw
    pitch = m.pi/180 * pitch
    Rx = np.matrix([[ 1, 0           , 0           ],
                   [ 0, m.cos(pitch),-m.sin(pitch)],
                   [ 0, m.sin(pitch), m.cos(pitch)]])
    Ry = np.matrix([[ m.cos(yaw), 0, m.sin(yaw)],
                   [ 0           , 1, 0           ],
                   [-m.sin(yaw), 0, m.cos(yaw)]])
    Rz = np.matrix([[ m.cos(roll), -m.sin(roll), 0 ],
                   [ m.sin(roll), m.cos(roll) , 0 ],
                   [ 0           , 0            , 1 ]])
    R = Rz * Ry * Rx
    R = np.array(R)
    return R

In [36]:
def GenerateModel(tvecs, f_x, f_y, cx, cy, roll, yaw, pitch):
    X = np.array([[markerLength / 2 + tvecs[0][0][1], -markerLength / 2 + tvecs[0][0][1], tvecs[0][0][2]],
                [markerLength / 2 + tvecs[0][0][1], markerLength / 2 + tvecs[0][0][1], tvecs[0][0][2]],
                [-markerLength / 2 + tvecs[0][0][1], markerLength / 2 + tvecs[0][0][1], tvecs[0][0][2]],
                [-markerLength / 2 + tvecs[0][0][1], -markerLength / 2 + tvecs[0][0][1], tvecs[0][0][2]]])

    R = np.array(eul2rotm(roll, yaw, pitch))
    K = [[f_x, 0, cx], [0, f_y, cy], [0, 0, 1]]
    K = np.array(K)

    Xt = np.dot(R, X.T)
    Xt = np.dot(K, Xt)
    Xt = Xt.T
    Xt = Xt/Xt[0][2]

    return Xt

In [38]:
roll, yaw, pitch = 0, 0, 0
A = GenerateModel(tvecs, camera_matrix[0][0], camera_matrix[1][1], camera_matrix[0][2], camera_matrix[1][2], roll, yaw, pitch)
print(aruco_coord)
print(A)

[[1408.0074 1903.363 ]
 [1552.9579 1912.8857]
 [1552.5175 2060.3716]
 [1409.5645 2049.5354]]
[[2.60610923e+03 1.88661576e+03 1.00000000e+00]
 [2.60610923e+03 2.03279286e+03 1.00000000e+00]
 [2.45993212e+03 2.03279286e+03 1.00000000e+00]
 [2.45993212e+03 1.88661576e+03 1.00000000e+00]]


In [ ]:
def projectionError(xinput, factor):
    error = xinput - GenerateModel(tvecs, camera_matrix[0][0], camera_matrix[1][1], camera_matrix[0][2], camera_matrix[1][2], factor[0], factor[1], factor[2])
    return error

def parameter(xinput, factor, bound_roll, bound_yaw, bound_pitch):


    def fxn(factor_x):  # xinput은 바꿔줘야함
        global estimated_X, error
        error = projectionError(xinput, factor_x)
        return error
    
    Optimization = least_squares(fxn, factor,
                                 bounds=[[bound_yaw[0], bound_pitch[0]], [bound_yaw[1], bound_pitch[1]]])
    
    return Optimization

In [ ]:
optimization=parameter(xinput, factor, bound_roll, bound_yaw, bound_pitch)
estimated_roll = optimization[0].x[0]
estimated_yaw = optimization[0].x[1]
estimated_pitch = optimization[0].x[2]